In [ ]:
#Path Define
csv_name = './SVM_Feature_1007.csv'

In [ ]:
from turtle import onrelease
from inference_cellpose import img_to_cellpose
from inference_cellpose import obj_detection
import cv2
import matplotlib.pyplot as plt
import numpy as np
import feret

def Compute_CellFeature(onemask):
    """
    Args:
        onemask : [width, height]
    Return:
        feature : [](area, length, circularity)
    """
    #白い領域の割合計算(0~1)
    w, h = onemask.shape
    #white_area = cv2.countNonZero(onemask)
    
    #Compute area, len, cir
    contours, _ = cv2.findContours(onemask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
    area = cv2.contourArea(contours[0], True)
    length = cv2.arcLength(contours[0], True)

    try:
        circularity = 4*np.pi*area/length/length
    except:
        circularity = 0
        print(length)
        print("length is 0")
    
    #Compute feret
    maxf, minf, minf90, maxf90 = feret.all(onemask)
    feret_ratio = maxf/minf

    return [abs(area), abs(length), abs(circularity), maxf,minf, minf90, minf90, maxf90, feret_ratio]

main

In [ ]:
from glob import glob
from tqdm import tqdm
import csv

folders = glob("train_0712_aug/*")
for cls, folder in enumerate(folders):
    files = glob(folder + "/*.png")

    for file in tqdm(files):
        masks = img_to_cellpose(file)
        masks, ids = obj_detection(masks, 1)#return [width, height, N]
        masks = masks.transpose(2,1,0)#(x,y,obj番号)→(obj番号,x,y)に転置．

        for i in range(len(masks)):
            feature = Compute_CellFeature(masks[i])
            feature = np.insert(feature, 0, int(cls))

            with open(csv_name, 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(feature)

In [ ]:
img = cv2.imread(img_path)
plt.matshow(img)
plt.matshow(masks[15]) 
print(masks.shape)